### Objective

In this notebook, we develop a global optimization routine to find the design that minimizes the weight while satisfying the geometrical and thermal constraints.

We consider multiple Qs as design specification.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import time
from deap import base, creator, tools, algorithms

from two_sources import thermal_distribution_maxT

### 1. Setup constants

In [2]:
df_Q = pd.read_csv('./dataset/Q_test_locations.csv')

In [3]:
Data = (25, 50e-3, 65e-3, 61.4e-3, 106e-3)
Tjmax = 175
c_module, d_module = 61.4e-3, 106e-3

d_min, d_max = 5e-3, 30e-3
b_min, b_max = 73.7e-3, 307e-3
L_min, L_max = 127.2e-3, 530e-3
c_min, c_max = 10e-3, 39e-3
L_duct_min, L_duct_max = 20e-3, 50e-3
n_min, n_max = 10, 50
t_min, t_max = 1e-3, b_max/n_min - 1e-3
Xc1_min, Xc1_max = c_module / 2, b_max - c_module / 2
Xc2_min, Xc2_max = c_module / 2, b_max - c_module / 2
Yc1_min, Yc1_max = d_module / 2, L_max - d_module / 2
Yc2_min, Yc2_max = d_module / 2, L_max - d_module / 2

#### 2. Problem definition

In [4]:
def objective(x):

    # Unpack design variables
    d, b, L, c, L_duct, n, t, xc1, yc1, xc2, yc2 = x

    # Calculate weight
    density_Al = 2700
    Fan_height = 40e-3
    Fan_Weight = 50.8e-3
    N_fan = np.ceil(b / Fan_height)

    # Weight calculation
    w = density_Al*(b*d*L+n*(c*t*L))+ Fan_Weight*N_fan
    
    return w

In [5]:
def constraint_maxT(x, Q1, Q2, Data, Tmax_threshold):    
    
    # Calculate Tmax
    input = np.hstack((Q1, Q2, x))
    Tmax, _ = thermal_distribution_maxT(input, Data)
    
    # Constraint Tmax to be less than or equal to Tmax_threshold
    if Tmax_threshold >= Tmax:
        return 0
    else:
        return 10000

In [ ]:
def constraint_geometric(x, c_module, d_module):

    # non_overlap_constraint

In [6]:
def non_overlap_constraint(x, c_module, d_module):

    # Unpack design variables
    d, b, L, c, L_duct, n, t, xc1, yc1, xc2, yc2 = x
    
    # Calculate distances
    xc_dist = abs(xc1 - xc2)  # Assuming these are Xc_first and Xc_second
    yc_dist = abs(yc1 - yc2)  # Assuming these are Yc_first and Yc_second
    
    # Constraints for non-overlapping
    if max(xc_dist - c_module, yc_dist - d_module) >= 0:
        return 0
    else:
        return 10000

In [7]:
def t_upper_bound(x):
    
    # Unpack design variables
    d, b, L, c, L_duct, n, t, xc1, yc1, xc2, yc2 = x

    if b / n - 1e-3 - t >= 0:
        return 0
    else:
        return 10000

In [8]:
def x1_upper_bound(x, c_module, d_module):
    
    # Unpack design variables
    d, b, L, c, L_duct, n, t, xc1, yc1, xc2, yc2 = x

    Xc_max = b - c_module / 2

    if Xc_max - xc1 >= 0:
        return 0
    else:
        return 10000

In [9]:
def x1_lower_bound(x, c_module, d_module):
    
    # Unpack design variables
    d, b, L, c, L_duct, n, t, xc1, yc1, xc2, yc2 = x

    Xc_min = c_module / 2

    if xc1 - Xc_min >= 0:
        return 0
    else:
        return 10000

In [10]:
def x2_upper_bound(x, c_module, d_module):
    
    # Unpack design variables
    d, b, L, c, L_duct, n, t, xc1, yc1, xc2, yc2 = x

    Xc_max = b - c_module / 2

    if Xc_max - xc2 >= 0:
        return 0
    else:
        return 10000

In [11]:
def x2_lower_bound(x, c_module, d_module):
    
    # Unpack design variables
    d, b, L, c, L_duct, n, t, xc1, yc1, xc2, yc2 = x

    Xc_min = c_module / 2

    if xc2 - Xc_min >= 0:
        return 0
    else:
        return 10000

In [12]:
def y1_upper_bound(x, c_module, d_module):
    
    # Unpack design variables
    d, b, L, c, L_duct, n, t, xc1, yc1, xc2, yc2 = x

    Yc_max = L - d_module / 2

    if Yc_max - yc1 >= 0:
        return 0
    else:
        return 10000

In [13]:
def y1_lower_bound(x, c_module, d_module):
    
    # Unpack design variables
    d, b, L, c, L_duct, n, t, xc1, yc1, xc2, yc2 = x

    Yc_min = d_module / 2

    if yc1 - Yc_min >= 0:
        return 0
    else:
        return 10000

In [14]:
def y2_upper_bound(x, c_module, d_module):
    
    # Unpack design variables
    d, b, L, c, L_duct, n, t, xc1, yc1, xc2, yc2 = x

    Yc_max = L - d_module / 2

    if Yc_max - yc2 >= 0:
        return 0
    else:
        return 10000

In [15]:
def y2_lower_bound(x, c_module, d_module):
    
    # Unpack design variables
    d, b, L, c, L_duct, n, t, xc1, yc1, xc2, yc2 = x

    Yc_min = d_module / 2

    if yc2 - Yc_min >= 0:
        return 0
    else:
        return 10000

In [16]:
class EvaluationCounter:
    def __init__(self, func):
        self.func = func
        self.counter = 0

    def __call__(self, *args):
        self.counter += 1
        return self.func(*args)

In [17]:
Tmax_contraint_counter = EvaluationCounter(constraint_maxT)

In [18]:
def make_eval_function(Q1, Q2, Data, c_module, d_module, Tmax_threshold, n_min, n_max):
    def eval_individual(individual):
        # Objective function value
        new_n = int(individual[5])
        individual[5] = max(min(new_n, n_max), n_min)
        objective_value = objective(individual)

        # Penalty for constraint violations
        if objective_value > 0:
            penalty = 0
        else:
            penalty = 10000
            
        try:
            penalty_t = Tmax_contraint_counter(individual, Q1, Q2, Data, Tmax_threshold)
        except Exception as e:
            penalty_t = 10000
            
        penalty += penalty_t
        penalty += non_overlap_constraint(individual, c_module, d_module)
        penalty += t_upper_bound(individual)
        penalty += x1_upper_bound(individual, c_module, d_module)
        penalty += x1_lower_bound(individual, c_module, d_module)
        penalty += x2_upper_bound(individual, c_module, d_module)
        penalty += x2_lower_bound(individual, c_module, d_module)
        penalty += y1_upper_bound(individual, c_module, d_module)
        penalty += y1_lower_bound(individual, c_module, d_module)
        penalty += y2_upper_bound(individual, c_module, d_module)
        penalty += y2_lower_bound(individual, c_module, d_module)

        return objective_value + penalty, 

    return eval_individual

In [19]:
def custom_mutate(individual, low, up, eta, indpb):
    # Clone the individual to avoid altering the original during the iteration
    # Mutate using mutPolynomialBounded for continuous variables
    _, = tools.mutPolynomialBounded(individual, low, up, eta, indpb)
    
    # Copy back the mutated values except for 'n' at index 5
    individual[5] = int(round(individual[5]))

    return individual,

In [20]:
def GA_run(Q1, Q2, num_gen=200):

    start = time.time()
    # Minimization problem. Adjust weights for maximization.
    creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMin)
    
    # Initialize DEAP tools for GA
    toolbox = base.Toolbox()
    
    # Individual and population setup
    # Adjust attribute generators according to your problem's variables
    toolbox.register("individual", tools.initCycle, creator.Individual, 
                     (lambda: random.uniform(d_min, d_max),
                      lambda: random.uniform(b_min, b_max),
                      lambda: random.uniform(L_min, L_max),
                      lambda: random.uniform(c_min, c_max),
                      lambda: random.uniform(L_duct_min, L_duct_max),
                      lambda: random.randint(n_min, n_max),
                      lambda: random.uniform(t_min, t_max),
                      lambda: random.uniform(Xc1_min, Xc1_max),
                      lambda: random.uniform(Yc1_min, Yc1_max),
                      lambda: random.uniform(Xc2_min, Xc2_max),
                      lambda: random.uniform(Yc2_min, Yc2_max)), n=1)
    
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    
    # Register genetic operators
    toolbox.register("evaluate", make_eval_function(Q1, Q2, Data, c_module, d_module, Tjmax, n_min, n_max))
    toolbox.register("mate", tools.cxBlend, alpha=0.5)
    # Update mutation registration to use the custom mutation function
    toolbox.register("mutate", tools.mutPolynomialBounded, 
                     low=[d_min, b_min, L_min, c_min, L_duct_min, n_min, t_min, Xc1_min, Yc1_min, Xc2_min, Yc2_min], 
                     up=[d_max, b_max, L_max, c_max, L_duct_max, n_max, t_max, Xc1_max, Yc1_max, Xc2_max, Yc2_max], 
                     eta=1.0, indpb=0.2)
    toolbox.register("select", tools.selTournament, tournsize=3)
    
    # Genetic algorithm parameters
    population_size = 20
    crossover_probability = 0.7
    mutation_probability = 0.5
    number_of_generations = num_gen
    
    # Initialize population
    pop = toolbox.population(n=population_size)
    
    # Statistics accumulator
    stats = tools.Statistics(key=lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("min", np.min)
    stats.register("max", np.max)

    hof_size = 1
    hall_of_fame = tools.HallOfFame(hof_size)
    
    # Evolutionary algorithm execution
    final_pop, logbook = algorithms.eaSimple(pop, toolbox, cxpb=crossover_probability, 
                                       mutpb=mutation_probability,
                                       ngen=number_of_generations, stats=stats, 
                                       halloffame=hall_of_fame, verbose=True)

    # Parse results
    best_ind = hall_of_fame[0]
    best_weight = best_ind.fitness.values[0]

    # Runtime calculation
    end = time.time()
    runtime = end - start

    return best_ind, best_weight, runtime, Tmax_contraint_counter.counter

In [21]:
best_designs = []
best_weights = []
runtimes = []
eval_nums = []
for i, (Q1, Q2) in enumerate(df_Q.to_numpy()[:10]):
    print(f"Processing {i+1}th iteration with Q1 ({Q1:.2f}) & Q2 ({Q2:2f}) combinations:")
    best_ind, best_weight, runtime, eval_num = GA_run(Q1, Q2, 300)
    best_designs.append(best_ind)
    best_weights.append(best_weight)
    runtimes.append(runtime)
    eval_nums.append(eval_num)

Processing 1th iteration with Q1 (324.55) & Q2 (273.426620) combinations:


D:\PE-Optimization\new_campaign\two_sources.py:67: RuntimeWarning: invalid value encountered in sqrt
  fRe_fd = 12 / (np.sqrt(EPS) * (1 + EPS) * (1 - 192 * EPS * np.tanh(np.pi / 2 / EPS) / np.pi ** 5))
D:\PE-Optimization\new_campaign\two_sources.py:69: RuntimeWarning: invalid value encountered in sqrt
  fapp = n * visc_air_K * np.sqrt(c * s) * fRe / V


gen	nevals	avg    	min    	max    
0  	20    	38507.8	10009.9	60017.1
1  	10    	27004.9	10001.5	60004.2


D:\PE-Optimization\new_campaign\two_sources.py:104: RuntimeWarning: invalid value encountered in sqrt
  eff_fin = np.tanh(np.sqrt(2 * h * (t + L) / lambda_HS / t / L) * c) / np.sqrt(2 * h * (t + L) / lambda_HS / t / L) / c


2  	13    	23505.3	10001.5	40008.9
3  	14    	17006.8	10001.5	39999.9
4  	17    	16507  	10007.7	40005.6
5  	19    	13009.1	10006.1	20018.1
6  	17    	12007.8	10002.6	30006.1
7  	18    	15006  	10002.5	50005.8
8  	15    	13505.1	10002.5	40003.6
9  	20    	18005  	10003.4	40003.3
10 	19    	11004.4	10002.6	20003  
11 	17    	13504.2	10002.4	40006.9
12 	17    	14003.8	10002.1	40007.3
13 	18    	9503.15	2.47956	30005.6
14 	20    	15503.8	8.20391	30004.6
15 	17    	10004.4	1.62759	30002.8
16 	17    	14003.4	2.84931	30003.6
17 	13    	10003.1	2.44039	30001.8
18 	17    	8502.86	2.19418	30001.8
19 	14    	10002.7	2.19418	40002.4
20 	15    	3502.94	2.04888	20005.6
21 	19    	7502.59	2.02952	30003.3
22 	20    	6002.52	2.05418	20002.6
23 	19    	8502.86	2.02477	40004.5
24 	17    	5002.36	1.87715	40001.7
25 	14    	4002.76	1.87715	30001.7
26 	18    	5002.51	1.87715	20002.2
27 	17    	10502.3	1.93926	30002.9
28 	18    	8502.2 	1.84664	30001.6
29 	18    	7502.22	1.82041	40003.3
30 	18    	3002.31	1

...

D:\PE-Optimization\new_campaign\two_sources.py:68: RuntimeWarning: invalid value encountered in sqrt
  fRe = np.sqrt(11.8336 * V / L / n / visc_air_K + fRe_fd ** 2)


132	20    	14500.8	0.605398	40001.6
133	15    	6501.06	0.605398	30000.5
134	17    	8001.21	0.605398	30002.2
135	11    	2501.05	0.645517	20000.6
136	17    	6500.86	0.617186	20001.9
137	15    	7500.76	0.544869	40000.4
138	19    	5000.78	0.554742	20000.8
139	13    	3501.06	0.623624	20004.1
140	19    	5001.07	0.604339	30000.2
141	19    	5000.79	0.6032  	30001  
142	17    	5500.71	0.604658	30000.6
143	15    	7500.78	0.604658	40000.5
144	20    	8000.74	0.60079 	20001.9
145	17    	6500.72	0.601075	30000.5
146	16    	5000.65	0.597772	20000.8
147	14    	1000.65	0.597772	10001  
148	20    	8000.93	0.594807	30000.9
149	16    	4500.76	0.594807	30002  
150	17    	4000.8 	0.59543 	20003.1
151	17    	5500.6 	0.593967	30000.5
152	15    	12000.9	0.593967	30001.1
153	18    	11000.9	0.596536	20001.9
154	16    	6500.82	0.591911	30000.4
155	17    	7500.72	0.591911	30001.4
156	17    	7000.7 	0.592408	30000.6
157	13    	6000.78	0.592408	20001.5
158	20    	3000.65	0.590667	10001.6
159	16    	4000.62	0.590168	

D:\PE-Optimization\new_campaign\two_sources.py:76: RuntimeWarning: invalid value encountered in sqrt
  fRe_fd_duct = 12 / (np.sqrt(EPS_duct) * (1 + EPS_duct) * (1 - 192 * EPS_duct * np.tanh(np.pi / 2 / EPS_duct) / np.pi ** 5))


212	20    	2500.67	0.489513	20000.5
213	19    	5000.56	0.48705 	30000.5
214	19    	7000.54	0.489513	20000.6
215	14    	4000.57	0.489513	20000.6
216	16    	6500.62	0.489013	30000.5
217	18    	6000.8 	0.489013	50002.6
218	16    	4500.6 	0.488704	20000.9
219	19    	4500.59	0.487793	30001.2
220	14    	2500.55	0.488122	20000.5
221	14    	1000.57	0.488605	10000.5
222	17    	5500.7 	0.488208	20000.8
223	20    	3500.66	0.488549	10001  
224	19    	6500.71	0.488412	20001.1
225	18    	4000.98	0.488412	30002.7
226	19    	4000.59	0.488361	20000.8
227	18    	3000.57	0.488412	20000.5
228	18    	7500.62	0.4871  	30002.8
229	17    	7500.54	0.4871  	20000.5
230	17    	6500.54	0.486929	20000.5
231	19    	9000.59	0.486832	30000.5
232	15    	8500.58	0.486832	30000.7
233	19    	9000.66	0.487211	30001  
234	15    	7500.76	0.487211	30000.9
235	15    	5500.57	0.473732	30000.5
236	15    	6000.59	0.473732	30002.4
237	14    	5500.48	0.473732	30000.5
238	16    	4500.53	0.473732	30000.3
239	17    	9000.84	0.473064	

KeyboardInterrupt: 

In [24]:
best_designs[0]

[-0.004803430814735736,
 0.07617469322684123,
 0.37850151884268285,
 0.014512971862034494,
 0.039349424695713064,
 26,
 0.0007091382210508011,
 0.04163678786001176,
 0.07363490769047978,
 0.032734131336991296,
 0.2701115643780079]

In [ ]:
df = pd.DataFrame({"Q1": df_Q.to_numpy()[:20, 0],
                  "Q2": df_Q.to_numpy()[:20, 1],
                   "weight": best_weights,
                  "Runtime": runtimes,
                  "Evalnum": eval_nums})
best_designs = np.array(best_designs)
for i, col in enumerate(['d', 'b', 'L', 'c', 'L_duct', 'n', 't', 'xc1', 'yc1', 'xc2', 'yc2']):
    df[col] = best_designs[:, i]

In [ ]:
df

In [ ]:
best_ind = hall_of_fame[0]
print(f"Best fitness:", best_ind.fitness.values[0])
columns = ['d', 'b', 'L', 'c', 'L_duct', 'n', 't', 'xc1', 'yc1', 'xc2', 'yc2']
df = pd.DataFrame({'optimized': best_ind})
df.index = columns
df

In [ ]:
# First Generation
best_ind = tools.selBest(final_pop, 1)[0]
print(f"Best fitness:", best_ind.fitness.values[0])
columns = ['d', 'b', 'L', 'c', 'L_duct', 'n', 't', 'xc1', 'yc1', 'xc2', 'yc2']
df = pd.DataFrame({'optimized': best_ind})
df.index = columns
df

#### Verification

In [ ]:
def verification_suite(design, Q1, Q2, Data, Tjmax, c_module, d_module):
    
    # Tmax constraint
    check = constraint_maxT(design, Q1, Q2, Data, Tjmax)
    if check >= 0:
        print(f"Tmax verification: True")
    else:
        print(f"Tmax exceeded by {check}")
    
    # Coordinates bounds
    check = x1_upper_bound(design, c_module, d_module)
    if check >= 0:
        print(f"x1 upper bound: True")
    else:
        print(f"x1 upper bound violated: {check}")

    check = x1_lower_bound(design, c_module, d_module)
    if check >= 0:
        print(f"x1 lower bound: True")
    else:
        print(f"x1 lower bound violated: {check}")

    check = x2_upper_bound(design, c_module, d_module)
    if check >= 0:
        print(f"x2 upper bound: True")
    else:
        print(f"x2 upper bound violated: {check}")

    check = x2_lower_bound(design, c_module, d_module)
    if check >= 0:
        print(f"x2 lower bound: True")
    else:
        print(f"x2 lower bound violated: {check}")

    check = y1_upper_bound(design, c_module, d_module)
    if check >= 0:
        print(f"y1 upper bound: True")
    else:
        print(f"y1 upper bound violated: {check}")

    check = y1_lower_bound(design, c_module, d_module)
    if check >= 0:
        print(f"y1 lower bound: True")
    else:
        print(f"y1 lower bound violated: {check}")

    check = y2_upper_bound(design, c_module, d_module)
    if check >= 0:
        print(f"y2 upper bound: True")
    else:
        print(f"y2 upper bound violated: {check}")

    check = y2_lower_bound(design, c_module, d_module)
    if check >= 0:
        print(f"y2 lower bound: True")
    else:
        print(f"y2 lower bound violated: {check}")

    
    # Non-overlapping bounds
    check = non_overlap_constraint(design, c_module, d_module)
    if check >= 0:
        print(f"Non-overlapping True")
    else:
        print(f"Non-overlapping violated: {check}")
    
    # t bounds
    check = t_upper_bound(design)
    if check >= 0:
        print(f"t bound verification True")
    else:
        print(f"t bound violated: {check}")

In [ ]:
verification_suite(best_ind, Q1, Q2, Data, Tjmax, c_module, d_module)